In [16]:
from neo4j import GraphDatabase
import matplotlib.pyplot as plt
import pandas as pd

In [17]:
uri = "bolt://neo4j:7687"
username = "neo4j"
password = "adminPass"

driver = GraphDatabase.driver(uri, auth= (username, password))
print( driver)

In [18]:
with driver.session() as session:
    query = """
    MATCH (a:Station)-[r:ROUTE]->(b:Station)
    RETURN a, r, b
    """
    records = session.run(query).data()  
    df_edges = pd.DataFrame(records)
    df_edges.head()


In [19]:
!pip -q install pyvis
from pyvis.network import Network

In [20]:
# 2) Connexion Neo4j
# - si ton notebook est sur ta machine : bolt://localhost:7687
# - si ton notebook est dans Docker : bolt://neo4j:7687

# 3) Récupérer toutes les routes (donc implicitement toutes les stations connectées)
query = """
MATCH (a:Station)-[r:ROUTE]->(b:Station)
RETURN a.station_id AS source, b.station_id AS target
"""

with driver.session() as session:
    rows = session.run(query).data()


print("Nombre de liens (ROUTE) :", len(rows))

# 4) Visualisation
net = Network(height="800px", width="100%", notebook=True, directed=True)

for row in rows:
    src = str(row["source"])
    dst = str(row["target"])
    net.add_node(src, label=src)
    net.add_node(dst, label=dst)
    net.add_edge(src, dst)

net.show("stations_graph.html")


Nombre de liens (ROUTE) : 1830
stations_graph.html


In [21]:
SOURCE_ID = "4004"
TARGET_ID = "4002"
WEIGHT_PROP = "risk" 

query = """
MATCH (a:Station {station_id: 10061}),
      (b:Station {station_id: 6017})
MATCH p = shortestPath((a)-[:ROUTE*..50]->(b))
RETURN [n IN nodes(p) | n.station_id] AS path;
"""

with driver.session() as s:
    rec = s.run(query, src=SOURCE_ID, dst=TARGET_ID).single()
    
print(rec["path"])


[10061, 10045, 6017]


In [24]:
WEIGHT_PROP = "risk"  # ou "risk_per_km"
SOURCE_ID = 20002       # <-- mets ton station_id
TARGET_ID = 4025       # <-- mets ton station_id

with driver.session() as session:
    # 1) Recréer une projection GDS propre (évite erreur "graph already exists")
    session.run("CALL gds.graph.drop('g', false) YIELD graphName RETURN graphName")
    session.run(f"""
        CALL gds.graph.project(
          'g',
          'Station',
          {{ ROUTE: {{ properties: ['{WEIGHT_PROP}'] }} }}
        )
    """)

    # 2) Dijkstra (chemin le moins risqué)
    rec = session.run(f"""
        MATCH (src:Station {{station_id:$src}}), (dst:Station {{station_id:$dst}})
        CALL gds.shortestPath.dijkstra.stream(
          'g',
          {{
            sourceNode: id(src),
            targetNode: id(dst),
            relationshipWeightProperty: '{WEIGHT_PROP}'
          }}
        )
        YIELD totalCost, nodeIds
        RETURN totalCost, nodeIds
    """, src=SOURCE_ID, dst=TARGET_ID).single()

    if rec is None:
        raise ValueError("Aucun chemin trouvé (vérifie les station_id ou la connectivité du graphe).")

    total_cost = rec["totalCost"]
    node_ids = rec["nodeIds"]

    # 3) Convertir nodeIds -> station_id et récupérer les arêtes du chemin avec leur poids
    rec2 = session.run(f"""
        WITH $nodeIds AS ids
        UNWIND range(0, size(ids)-2) AS i
        MATCH (a) WHERE id(a) = ids[i]
        MATCH (b) WHERE id(b) = ids[i+1]
        MATCH (a)-[r:ROUTE]->(b)
        RETURN a.station_id AS source,
               b.station_id AS target,
               r.{WEIGHT_PROP} AS w
        ORDER BY i
    """, nodeIds=node_ids).data()

print("Poids total du chemin (", WEIGHT_PROP, ") =", total_cost)
print("Arêtes du chemin :")
for e in rec2:
    print(f"  {e['source']} -> {e['target']}  ({WEIGHT_PROP}={e['w']:.3f})")

# 4) Visualiser le chemin (seulement le chemin)
net = Network(height="500px", width="100%", notebook=True, directed=True)

for e in rec2:
    src, dst, w = str(e["source"]), str(e["target"]), float(e["w"])
    net.add_node(src, label=src)
    net.add_node(dst, label=dst)
    net.add_edge(src, dst, label=f"{w:.2f}", title=f"{WEIGHT_PROP}={w:.3f}")

net.show("risk_path.html")


Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. id is deprecated. It is replaced by elementId or consider using an application-generated id.', position=<SummaryInputPosition line=6, column=25, offset=178>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 178, 'line': 6, 'column': 25}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "\n        MATCH (src:Station {station_id:$src}), (dst:Station {station_id:$dst})\n        CALL gds.shortestPath.dijkstra.stream(\n          'g',\n          {\n            sourceNode: id(src),\n            targetNode: id(dst),\n            relationshipWeightProperty: 'risk'\n          }\n        )\n        YIELD

Poids total du chemin ( risk ) = 4.5
Arêtes du chemin :
  20002 -> 21001  (risk=1.500)
  21001 -> 7006  (risk=1.000)
  7006 -> 7015  (risk=0.000)
  7015 -> 16002  (risk=0.000)
  16002 -> 17003  (risk=0.000)
  17003 -> 15002  (risk=0.000)
  15002 -> 15001  (risk=0.000)
  15001 -> 14001  (risk=0.500)
  14001 -> 10115  (risk=0.500)
  10115 -> 10087  (risk=0.000)
  10087 -> 10117  (risk=0.000)
  10117 -> 10058  (risk=0.000)
  10058 -> 10086  (risk=0.000)
  10086 -> 10091  (risk=0.000)
  10091 -> 10044  (risk=0.000)
  10044 -> 10001  (risk=0.000)
  10001 -> 10103  (risk=0.000)
  10103 -> 10124  (risk=0.000)
  10124 -> 10120  (risk=0.000)
  10120 -> 10055  (risk=0.000)
  10055 -> 10053  (risk=0.000)
  10053 -> 6042  (risk=0.000)
  6042 -> 6037  (risk=0.000)
  6037 -> 4012  (risk=0.500)
  4012 -> 4005  (risk=0.500)
  4005 -> 4022  (risk=0.000)
  4022 -> 4004  (risk=0.000)
  4004 -> 4007  (risk=0.000)
  4007 -> 4025  (risk=0.000)
risk_path.html
